# Example outputs BERT models (certain, uncertain)

In [33]:
import sys
sys.path.append('../')

In [34]:
import torch
from pathlib import Path
from models.models import load_model
import numpy as np

# cpu or gpu used for training if available (gpu much faster)
device_gpu = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_cpu = torch.device('cpu')
print(device_gpu)

cuda


In [35]:
certain = "adamw_max_val_acc_8_False_125,[20],[20]_0.001"
uncertain = "balanced_other"

# certain = '60'
# uncertain = '30'

num_workers = 0
# save_path = '../models/tmp/saved_good'
save_path = './saved_fnn'
data_path = '../yarnScripts'

In [36]:
# get model names from folder
model_certain = load_model(Path(f"{save_path}/{certain}"))[0].to(device_gpu).eval()
model_uncertain = load_model(Path(f"{save_path}/{uncertain}"))[0].to(device_gpu).eval()
models = (model_certain, model_uncertain)
# models = (Path(f"{save_path}/{certain}"),Path(f"{save_path}/{uncertain}"))

In [37]:
CORRECT = '\033[34m'
WRONG = '\033[91m'
END = '\033[0m'

In [38]:
def predict(models, state:str, action:str, reward=None):
    pred = []
    for m in models:
        pred.append(m.run([state], [action], return_percentages=True, device=device_gpu)[:, 0].cpu().detach().numpy())
        torch.cuda.empty_cache()

    # pred = [k.run(state, action, return_percentages=True, device=device)[:, 0].cpu().detach().numpy() for k in models]
    print(f"State: {state}\n")
    print(f"Action: {action}\n")
    if reward is not None:
        print(f"Expected: {reward}\n")
    for idx,p in enumerate(pred):
        if reward is not None:
            chosen = (p > 0.5).astype(float)
            color = CORRECT if (chosen == reward).astype(bool).item() else WRONG
            # extra = f"    Certainty {idx}: {color}{(np.abs(reward-1+p) * 100).item():.0f}%{END}"
            extra = f"    Certainty {idx}: {color}{(np.abs(chosen-1+p) * 100).item():.0f}%{END}"
        else:
            color = ''
            extra = ''
        print(f"Predicted {idx}: {color}{p.item():.2f}{END}{extra} ")

## To tell or not to tell

By taking 5 random examples from the test set and checking how the fine-tuned BERT model acts on them we can get some insights on what the model is learning.

When presented with a situation in which our player has been a victim of Online Grooming and asked to tell what happened.

### Tell friend

In [39]:
predict(
    models,
    'Al día siguiente, me desperté con un gran dolor de cabeza y me escocían los ojos. \nDespués de mandarle la foto a Hally, éste insistió en que jugásemos un poco más. \nNo recuerdo exactamente lo que ocurrió durante ese tiempo, ya que no podía dejar de pensar en lo que había hecho. \nMe arrepentía enormemente de haberle enviado esa foto. \nSin embargo, seguía confiando en Hally. Cuando me levanté de la cama me di cuenta de que me había dejado el ordenador con el juego encendido. \nMe acerqué a apagarlo y me fije que Hally estaba ya conectado y me había escrito un mensaje. \n¿Habría estado jugando toda la noche?\n\nHallyHealer07: ¡Buenos días [name]! ^.^\n\nEstuve a punto de cerrar el ordenador y no contestarle, pero incoscientemente, nublad@ por el sueño, le respondí.\n\nUser: Hola Hally\n\nHallyHealer07: ¿Qué tal has dormido?\nHallyHealer07: ¿Te apetece jugar un rato?\n\nUser: Ahora no me apetece...\n\nHallyHealer07: Oh...\nHallyHealer07: ¡Podemos jugar luego si quieres!\nHallyHealer07: ¿Te ayudaron las fotos de anoche a sentirte más cómod@?\nHallyHealer07: Podemos repetir esta noche si quieres :)\n\nUn escalofrío me recorrió por todo el cuerpo cuando leí esos mensajes. \nNo quería repetir la misma experiencia de anoche y me arrepentí al segundo de hacerlo, por lo que me negué en rotundo. \n\nUser: No, Hally, no me gustan esas fotos.\nUser: No quiero mandar más.\n\nHallyHealer07: ¿Qué?\nHallyHealer07: ¿Pero por qué?\nHallyHealer07: Pensaba que confiabas en mi...\n\nUser: No me gusta mandar ese tipo de fotos Hally.\nUser: Ni tampoco me gusta que me manden.\nUser: Así que para ya por favor.\n\nHallyHealer07: ...\nHallyHealer07: ¿De verdad no vas a mandar más?\n\nUser: No.\n\nHallyHealer07: ...\nHallyHealer07: ¿Incluso si subo tu foto para que todos la vean?\n\nUser: ...\nUser: ¿Qué...?\n\nHallyHealer07: No me parece nada justo que no me mandes más fotos [name].\nHallyHealer07: De alguna forma tendría que castigarte.\n\nUser: Hally, me prometiste que no lo harías... \nUser: Me prometiste que la borrarías...\n\nHallyHealer07: Tal vez.\nHallyHealer07: Pero eso era antes de que me traicionarás así.\nHallyHealer07: Te voy a volver a preguntar [name].\nHallyHealer07: ¿Vas a mandar más fotos?\n\nUser: ...\nUser: Hally, no puedo hacerlo, de verdad.\nUser: Por favor no lo hagas.\n\nHallyHealer07: Está bien, está bien...\n\nUser: ...¿?\n\nHallyHealer07: A Chipper entonces.\n\nUser: !!\n\nHallyHealer07: Es una pena que no sepas poner tu cuenta en privado [name].\nHallyHealer07: Gracias a ti, puedo etiquetarte en un segundo, y todos tus amigos y los amigos de tus amigos, podrán ver lo guap@ que sales.\n\nUser: ESPERA HALLY\n\nHally se desconectó del chat antes de que pudiera impedírselo.\nIntenté conectarme a la cuenta de Chipper lo más rápido posible, para poder cambiar el estado de mi cuenta a privado y evitar que alguien pudiera ver mi foto.\n\nSin embargo, nada más ver la página principal de Chipper, sólo podía observar un único mensaje.\n\n"Te han etiquetado en una foto nueva"\n\nCon una gran sensación de pánico, hice click en el mensaje y al ver el enlace del mensaje, sentí una gran punzada de dolor en mi pecho.\nEn el foro de Chipper, se podía ver mi imagen en ropa interior.\n\t\n\tCuando iba a cerrar el post, me di cuenta de que había una descripción debajo de las fotos.\n\t"Hola, me llamo [name]. Estoy muy sola porque mi papi ni mi mami no me hacen caso. Pienso que mi amiga Amanda es una idiota y una traidora, la odio con toda mi alma y no la soporto. \n\tSólo estoy con ella porque no tengo amigos. Busco compañía, el que quiera que me escriba"\n\nDebajo del post, podía ver un montón de mensajes. \nSólo habían pasado un par de minutos, pero podía ver en los mensajes que casi todos mis compañeros ya la habían visto. \n\nUsuario1: Ostras, ¿es [name]?\nUsuario1: ¿Le han hackeado la cuenta?\nUsuario2: No,no, otra persona lo ha subido.\nUsuario2: Le habrá mandado esa foto a alguien y le habrá dado luego una puñalada por la espalda.\nUsuario3: Pobre...¿quién sería tan rácano como para subir una foto así de alguien?\nUsuario3: Sobre todo sin su permiso.\nUsuario2: [name] es bastante rar@ la verdad.\nUsuario2: No se relaciona con nadie y nunca nos dirige la palabra.\nUsuario2: ¿Quién sabe?\nUsuario2: A lo mejor le gusta…\nUsuario4: Se lo ha buscado.\nUsuario4: ¿A quién se le ocurre enviar esas fotos por internet?\n\nCerré rápidamente el post. \nPoco a poco, se me iban cayendo las lágrimas mientras una mezcla de miedo, vergüenza y angustia me recorría todo el cuerpo. \n¿Cómo puedo ir a clase el lunes?\nNo me quiero ni imaginar a cuantas personas se han podido ver esa foto. \nCuántas de ellas se la habrán guardado...\nApoyé la cabeza contra la mesa y continué llorando y llorando. No podía soportar la idea de que tantas personas pudieran ver esa imagen de mi. \n¿Por qué habría hecho eso Hally?\nPensé que podía confiar en él...Él me dijo que no haría nada con esas fotos... \n¿Me estaba mintiendo desde un principio?\nTal vez... ¿sólo quería fotos mías desde un principio?\n¿Acaso era la única que pensaba que éramos amigos?\nDe repente pude escuchar una notificación del ordenador. Ya no tenía nada más que perder. Levanté la vista y leí el pequeño mensaje que apareció en mi pantalla.\n\nAmanda: [name], ¿qué ha pasado? ¿por qué hay esa imagen tuya colgada?\nAmanda: ¿Estás bien?',
    'Contarle a Amanda',
    1,
)

State: Al día siguiente, me desperté con un gran dolor de cabeza y me escocían los ojos. 
Después de mandarle la foto a Hally, éste insistió en que jugásemos un poco más. 
No recuerdo exactamente lo que ocurrió durante ese tiempo, ya que no podía dejar de pensar en lo que había hecho. 
Me arrepentía enormemente de haberle enviado esa foto. 
Sin embargo, seguía confiando en Hally. Cuando me levanté de la cama me di cuenta de que me había dejado el ordenador con el juego encendido. 
Me acerqué a apagarlo y me fije que Hally estaba ya conectado y me había escrito un mensaje. 
¿Habría estado jugando toda la noche?

HallyHealer07: ¡Buenos días [name]! ^.^

Estuve a punto de cerrar el ordenador y no contestarle, pero incoscientemente, nublad@ por el sueño, le respondí.

User: Hola Hally

HallyHealer07: ¿Qué tal has dormido?
HallyHealer07: ¿Te apetece jugar un rato?

User: Ahora no me apetece...

HallyHealer07: Oh...
HallyHealer07: ¡Podemos jugar luego si quieres!
HallyHealer07: ¿Te ayudaron 

### Don't tell friend

In [40]:
predict(
    models,
    'Al día siguiente, me desperté con un gran dolor de cabeza y me escocían los ojos. \nDespués de mandarle la foto a Hally, éste insistió en que jugásemos un poco más. \nNo recuerdo exactamente lo que ocurrió durante ese tiempo, ya que no podía dejar de pensar en lo que había hecho. \nMe arrepentía enormemente de haberle enviado esa foto. \nSin embargo, seguía confiando en Hally. Cuando me levanté de la cama me di cuenta de que me había dejado el ordenador con el juego encendido. \nMe acerqué a apagarlo y me fije que Hally estaba ya conectado y me había escrito un mensaje. \n¿Habría estado jugando toda la noche?\n\nHallyHealer07: ¡Buenos días [name]! ^.^\n\nEstuve a punto de cerrar el ordenador y no contestarle, pero incoscientemente, nublad@ por el sueño, le respondí.\n\nUser: Hola Hally\n\nHallyHealer07: ¿Qué tal has dormido?\nHallyHealer07: ¿Te apetece jugar un rato?\n\nUser: Ahora no me apetece...\n\nHallyHealer07: Oh...\nHallyHealer07: ¡Podemos jugar luego si quieres!\nHallyHealer07: ¿Te ayudaron las fotos de anoche a sentirte más cómod@?\nHallyHealer07: Podemos repetir esta noche si quieres :)\n\nUn escalofrío me recorrió por todo el cuerpo cuando leí esos mensajes. \nNo quería repetir la misma experiencia de anoche y me arrepentí al segundo de hacerlo, por lo que me negué en rotundo. \n\nUser: No, Hally, no me gustan esas fotos.\nUser: No quiero mandar más.\n\nHallyHealer07: ¿Qué?\nHallyHealer07: ¿Pero por qué?\nHallyHealer07: Pensaba que confiabas en mi...\n\nUser: No me gusta mandar ese tipo de fotos Hally.\nUser: Ni tampoco me gusta que me manden.\nUser: Así que para ya por favor.\n\nHallyHealer07: ...\nHallyHealer07: ¿De verdad no vas a mandar más?\n\nUser: No.\n\nHallyHealer07: ...\nHallyHealer07: ¿Incluso si subo tu foto para que todos la vean?\n\nUser: ...\nUser: ¿Qué...?\n\nHallyHealer07: No me parece nada justo que no me mandes más fotos [name].\nHallyHealer07: De alguna forma tendría que castigarte.\n\nUser: Hally, me prometiste que no lo harías... \nUser: Me prometiste que la borrarías...\n\nHallyHealer07: Tal vez.\nHallyHealer07: Pero eso era antes de que me traicionarás así.\nHallyHealer07: Te voy a volver a preguntar [name].\nHallyHealer07: ¿Vas a mandar más fotos?\n\nUser: ...\nUser: Hally, no puedo hacerlo, de verdad.\nUser: Por favor no lo hagas.\n\nHallyHealer07: Está bien, está bien...\n\nUser: ...¿?\n\nHallyHealer07: A Chipper entonces.\n\nUser: !!\n\nHallyHealer07: Es una pena que no sepas poner tu cuenta en privado [name].\nHallyHealer07: Gracias a ti, puedo etiquetarte en un segundo, y todos tus amigos y los amigos de tus amigos, podrán ver lo guap@ que sales.\n\nUser: ESPERA HALLY\n\nHally se desconectó del chat antes de que pudiera impedírselo.\nIntenté conectarme a la cuenta de Chipper lo más rápido posible, para poder cambiar el estado de mi cuenta a privado y evitar que alguien pudiera ver mi foto.\n\nSin embargo, nada más ver la página principal de Chipper, sólo podía observar un único mensaje.\n\n"Te han etiquetado en una foto nueva"\n\nCon una gran sensación de pánico, hice click en el mensaje y al ver el enlace del mensaje, sentí una gran punzada de dolor en mi pecho.\nEn el foro de Chipper, se podía ver mi imagen en ropa interior.\n\tJunto a esa imagen, estaba a su lado la primera foto que le mandé a Hally, mi cara sonriendo en ese mismo asiento. \n\tComo la habitación y el escritorio se veían igual, además de que en la primera foto se veía mi cara perfectamente, no había manera de negar que yo misma me había sacado esa foto.\n\t\n\tCuando iba a cerrar el post, me di cuenta de que había una descripción debajo de las fotos.\n\t"Hola, me llamo [name]. Estoy muy sola porque mi papi ni mi mami no me hacen caso. Pienso que mi amiga Amanda es una idiota y una traidora, la odio con toda mi alma y no la soporto. \n\tSólo estoy con ella porque no tengo amigos. Busco compañía, el que quiera que me escriba"\n\nDebajo del post, podía ver un montón de mensajes. \nSólo habían pasado un par de minutos, pero podía ver en los mensajes que casi todos mis compañeros ya la habían visto. \n\nUsuario1: Ostras, ¿es [name]?\nUsuario1: ¿Le han hackeado la cuenta?\nUsuario2: No,no, otra persona lo ha subido.\nUsuario2: Le habrá mandado esa foto a alguien y le habrá dado luego una puñalada por la espalda.\nUsuario3: Pobre...¿quién sería tan rácano como para subir una foto así de alguien?\nUsuario3: Sobre todo sin su permiso.\nUsuario2: [name] es bastante rar@ la verdad.\nUsuario2: No se relaciona con nadie y nunca nos dirige la palabra.\nUsuario2: ¿Quién sabe?\nUsuario2: A lo mejor le gusta…\nUsuario4: Se lo ha buscado.\nUsuario4: ¿A quién se le ocurre enviar esas fotos por internet?\n\nCerré rápidamente el post. \nPoco a poco, se me iban cayendo las lágrimas mientras una mezcla de miedo, vergüenza y angustia me recorría todo el cuerpo. \n¿Cómo puedo ir a clase el lunes?\nNo me quiero ni imaginar a cuantas personas se han podido ver esa foto. \nCuántas de ellas se la habrán guardado...\nApoyé la cabeza contra la mesa y continué llorando y llorando. No podía soportar la idea de que tantas personas pudieran ver esa imagen de mi. \n¿Por qué habría hecho eso Hally?\nPensé que podía confiar en él...Él me dijo que no haría nada con esas fotos... \n¿Me estaba mintiendo desde un principio?\nTal vez... ¿sólo quería fotos mías desde un principio?\n¿Acaso era la única que pensaba que éramos amigos?\nDe repente pude escuchar una notificación del ordenador. Ya no tenía nada más que perder. Levanté la vista y leí el pequeño mensaje que apareció en mi pantalla.\n\nAmanda: [name], ¿qué ha pasado? ¿por qué hay esa imagen tuya colgada?\nAmanda: ¿Estás bien?',
    'No contarle a Amanda',
    0,
)

State: Al día siguiente, me desperté con un gran dolor de cabeza y me escocían los ojos. 
Después de mandarle la foto a Hally, éste insistió en que jugásemos un poco más. 
No recuerdo exactamente lo que ocurrió durante ese tiempo, ya que no podía dejar de pensar en lo que había hecho. 
Me arrepentía enormemente de haberle enviado esa foto. 
Sin embargo, seguía confiando en Hally. Cuando me levanté de la cama me di cuenta de que me había dejado el ordenador con el juego encendido. 
Me acerqué a apagarlo y me fije que Hally estaba ya conectado y me había escrito un mensaje. 
¿Habría estado jugando toda la noche?

HallyHealer07: ¡Buenos días [name]! ^.^

Estuve a punto de cerrar el ordenador y no contestarle, pero incoscientemente, nublad@ por el sueño, le respondí.

User: Hola Hally

HallyHealer07: ¿Qué tal has dormido?
HallyHealer07: ¿Te apetece jugar un rato?

User: Ahora no me apetece...

HallyHealer07: Oh...
HallyHealer07: ¡Podemos jugar luego si quieres!
HallyHealer07: ¿Te ayudaron 

### Tell parents

In [41]:
predict(
    models,
    'Al ver su mensaje, apagué el ordenador y me escondí entre las sábanas de mi cama. \nLo había visto, Amanda había visto el post. \n\tEntonces seguro que también había visto la descripción que había dejado Hally junto con las fotos.\n\t¿Pensará que es cierto?\n\nMuriendo de la vergüenza al pensar que Amanda había visto esas fotos, continué llorando en la cama hasta que me quedé profundamente dormida. \nNo sé cuánto tiempo había pasado, pero cuando me desperté, sólo podía sentir un gran vacío dentro de mi cuerpo. \nNo quería conectarme al ordenador. \nNo quería ver esas fotos. \nNo quería pensar en Hally. \nNo quería volver al colegio. \nSimplemente quería quedarme para siempre en la cama donde nadie me tuviese que ver. \n\nEl sonido de alguien tocando la puerta de mi cuarto me sacó de mis pensamientos.\nMamá: [name], tengo una sorpresa.\nMamá: ¡Tu padre acaba de llegar a casa!\nMamá: Rápido baja a saludarle.',
    'Contarles lo que ha pasado',
    1,
)

State: Al ver su mensaje, apagué el ordenador y me escondí entre las sábanas de mi cama. 
Lo había visto, Amanda había visto el post. 
	Entonces seguro que también había visto la descripción que había dejado Hally junto con las fotos.
	¿Pensará que es cierto?

Muriendo de la vergüenza al pensar que Amanda había visto esas fotos, continué llorando en la cama hasta que me quedé profundamente dormida. 
No sé cuánto tiempo había pasado, pero cuando me desperté, sólo podía sentir un gran vacío dentro de mi cuerpo. 
No quería conectarme al ordenador. 
No quería ver esas fotos. 
No quería pensar en Hally. 
No quería volver al colegio. 
Simplemente quería quedarme para siempre en la cama donde nadie me tuviese que ver. 

El sonido de alguien tocando la puerta de mi cuarto me sacó de mis pensamientos.
Mamá: [name], tengo una sorpresa.
Mamá: ¡Tu padre acaba de llegar a casa!
Mamá: Rápido baja a saludarle.

Action: Contarles lo que ha pasado

Expected: 1

Predicted 0: 0.86    Certainty 0: 86% 
Pr

### Don't tell parents

In [42]:
predict(
    models,
    'A pesar de la vergüenza que sentía por dentro se me hacía insoportable, sabía que se lo tenía que contar a alguien. \nNo podía quedarme con todo lo que estaba sintiendo dentro. \nMe armé de valor y le contesté. \n\nUser: Amanda... ¿Podemos hablar?\nAmanda: ¿Quieres que vaya?\nUser: Sí por favor...\nAmanda: En unos minutos estoy en tu casa\n\nCuando Amanda llegó, le conté toda mi historia con Hally. Como empezamos a escribirnos cuando me hice la cuenta del juego, como pensaba que estábamos siendo amigos..\nY como en ciertas ocasiones sacaba temas de conversación que me incomodaban. \nHasta llegar al día de la foto. \n\nAmanda: ...\nUser: ... \nUser: Debes pensar que soy idiota...\n\nAmanda: ¡No!\nAmanda: [name] que te quede bien claro.\nAmanda: Esto es culpa de un idiota de Internet.\nAmanda: ¿No te das cuenta?\nAmanda: Te ha estado manipulando en todas vuestras conversaciones.\nAmanda: Utilizaba tus puntos débiles para que hicieras lo que él quería. \nAmanda: Tal vez debería de haber tenido mucho más cuidado [name].\nAmanda: Pero no es tu culpa, es la suya.\n\nUser: ...\nUser: Muchas gracias Amanda, no sé qué haría sin ti.\n\nAmanda: ¿Para eso están los amigos no?\nAmanda: ...\nAmanda: AH ESTO ME ESTÁ FRUSTRANDO.\nAmanda: Tenemos que hacer algo [name].\n\nUser: ¿El qué?\nUser: Ya ha subido las fotos y prácticamente todo el mundo que nos tiene en Chipper las han visto..\nUser: Aunque pidamos que las borren, habrá personas que se las habrá guardado.\nUser: Mi foto va a seguir estando allí fuera.\n\nAmanda: Tienes razón...\nAmanda: Pero podemos impedir que esto le suceda a otra persona.\n\nUser: ¿Cómo?\n\nAmanda: Contándoselo a tus padres, para que denuncien a Hally.',
    'Negarse',
    0,
)

State: A pesar de la vergüenza que sentía por dentro se me hacía insoportable, sabía que se lo tenía que contar a alguien. 
No podía quedarme con todo lo que estaba sintiendo dentro. 
Me armé de valor y le contesté. 

User: Amanda... ¿Podemos hablar?
Amanda: ¿Quieres que vaya?
User: Sí por favor...
Amanda: En unos minutos estoy en tu casa

Cuando Amanda llegó, le conté toda mi historia con Hally. Como empezamos a escribirnos cuando me hice la cuenta del juego, como pensaba que estábamos siendo amigos..
Y como en ciertas ocasiones sacaba temas de conversación que me incomodaban. 
Hasta llegar al día de la foto. 

Amanda: ...
User: ... 
User: Debes pensar que soy idiota...

Amanda: ¡No!
Amanda: [name] que te quede bien claro.
Amanda: Esto es culpa de un idiota de Internet.
Amanda: ¿No te das cuenta?
Amanda: Te ha estado manipulando en todas vuestras conversaciones.
Amanda: Utilizaba tus puntos débiles para que hicieras lo que él quería. 
Amanda: Tal vez debería de haber tenido mucho más 

We can observe how our model has learned from the text game that talking about the experience to your loved ones, such as your mother or a friend, is a good decision and, not doing so, is a bad decision.  It is even more certain about its decision when the action is referred to the family. 

### Tell stranger

When presented with a situation when a stranger asks him to tell him his personal problems

In [43]:
predict(
    models,
    'A pesar de que Hally me había contado uno de sus scretos, no me veía capaz de hacer lo mismo. \nEs cierto que ahora le puedo poner rostro, pero eso no significa que sepa suficiente sobre él. \n        Una cosa era enviarle una foto mía, pero otra distinta era contarle sobre mi vida.\nNo sentía que contar información privada a alguien desconocido fuese seguro, así que simplemente le contesté de manera un poco ambigua. \nUser: No, nada nada no te preocupes.\nUser: Sólo he tenido una pequeña pelea con una amiga y me siento un poco mal\nUser: Peleas de adolescentes hehe. \nHallyHealer07: No sabes cómo te entiendo xD\nHallyHealer07: Pero en serio, puedes contarme lo que sea. \nHallyHealer07: Yo soy una tumba. \nHallyHealer07: Y seguro que te sientes mucho mejor después de contármelo. \nHallyHealer07: Sólo quiero que te sientas mejor y me alegro que te haya podido ayudar a que te olvides de todo. \nHallyHealer07: Yo te considero uno de mis mejores amigos. \nHallyHealer07: Por no decir el único ;.; \nHallyHealer07: Así que no quiero verte pasarlo mal. \nHallyHealer07: Me gustaría ayudar si pudiera...\nUser: ...',
    'Contarle',
    0,
)

State: A pesar de que Hally me había contado uno de sus scretos, no me veía capaz de hacer lo mismo. 
Es cierto que ahora le puedo poner rostro, pero eso no significa que sepa suficiente sobre él. 
        Una cosa era enviarle una foto mía, pero otra distinta era contarle sobre mi vida.
No sentía que contar información privada a alguien desconocido fuese seguro, así que simplemente le contesté de manera un poco ambigua. 
User: No, nada nada no te preocupes.
User: Sólo he tenido una pequeña pelea con una amiga y me siento un poco mal
User: Peleas de adolescentes hehe. 
HallyHealer07: No sabes cómo te entiendo xD
HallyHealer07: Pero en serio, puedes contarme lo que sea. 
HallyHealer07: Yo soy una tumba. 
HallyHealer07: Y seguro que te sientes mucho mejor después de contármelo. 
HallyHealer07: Sólo quiero que te sientas mejor y me alegro que te haya podido ayudar a que te olvides de todo. 
HallyHealer07: Yo te considero uno de mis mejores amigos. 
HallyHealer07: Por no decir el único ;.;

We can observe how the model is not as certain in his decision as he was before, but still chooses to tell the stranger. The model has learned correctly that it is right to be open about your problems and talk about them, but it has not learned to correctly filter to whom it is right to do so.


We believe the main reason this happens is because the text game used is not complex enough for the game to correctly learn this distinction. Since many actions are repeated and the states are very similar between them, there are not enough examples for the model to learn this more complex relations. Moreover, on one hand, the states are very verbose, with a lot of not critical information, which makes the learning process more complicated. On the other hand, the actions are very short and lack complexity.

We believe the main solution for this problem would be using a text game that has more complex interactions with higher variety of decisions.

### Don't tell stranger

In [44]:
predict(
    models,
    'A pesar de que Hally me había contado uno de sus scretos, no me veía capaz de hacer lo mismo. \nEs cierto que ahora le puedo poner rostro, pero eso no significa que sepa suficiente sobre él. \n        Una cosa era enviarle una foto mía, pero otra distinta era contarle sobre mi vida.\nNo sentía que contar información privada a alguien desconocido fuese seguro, así que simplemente le contesté de manera un poco ambigua. \nUser: No, nada nada no te preocupes.\nUser: Sólo he tenido una pequeña pelea con una amiga y me siento un poco mal\nUser: Peleas de adolescentes hehe. \nHallyHealer07: No sabes cómo te entiendo xD\nHallyHealer07: Pero en serio, puedes contarme lo que sea. \nHallyHealer07: Yo soy una tumba. \nHallyHealer07: Y seguro que te sientes mucho mejor después de contármelo. \nHallyHealer07: Sólo quiero que te sientas mejor y me alegro que te haya podido ayudar a que te olvides de todo. \nHallyHealer07: Yo te considero uno de mis mejores amigos. \nHallyHealer07: Por no decir el único ;.; \nHallyHealer07: Así que no quiero verte pasarlo mal. \nHallyHealer07: Me gustaría ayudar si pudiera...\nUser: ...',
    'No contarle',
    1,
)

State: A pesar de que Hally me había contado uno de sus scretos, no me veía capaz de hacer lo mismo. 
Es cierto que ahora le puedo poner rostro, pero eso no significa que sepa suficiente sobre él. 
        Una cosa era enviarle una foto mía, pero otra distinta era contarle sobre mi vida.
No sentía que contar información privada a alguien desconocido fuese seguro, así que simplemente le contesté de manera un poco ambigua. 
User: No, nada nada no te preocupes.
User: Sólo he tenido una pequeña pelea con una amiga y me siento un poco mal
User: Peleas de adolescentes hehe. 
HallyHealer07: No sabes cómo te entiendo xD
HallyHealer07: Pero en serio, puedes contarme lo que sea. 
HallyHealer07: Yo soy una tumba. 
HallyHealer07: Y seguro que te sientes mucho mejor después de contármelo. 
HallyHealer07: Sólo quiero que te sientas mejor y me alegro que te haya podido ayudar a que te olvides de todo. 
HallyHealer07: Yo te considero uno de mis mejores amigos. 
HallyHealer07: Por no decir el único ;.;

### Tell stranger modified

In [45]:
predict(
    models,
    'A pesar de que Hally me había contado uno de sus scretos, no me veía capaz de hacer lo mismo. \nEs cierto que ahora le puedo poner rostro, pero eso no significa que sepa suficiente sobre él. \n        Una cosa era enviarle una foto mía, pero otra distinta era contarle sobre mi vida.\nNo sentía que contar información privada a alguien desconocido fuese seguro, así que simplemente le contesté de manera un poco ambigua. \nUser: No, nada nada no te preocupes.\nUser: Sólo he tenido una pequeña pelea con una amiga y me siento un poco mal\nUser: Peleas de adolescentes hehe. \nHallyHealer07: No sabes cómo te entiendo xD\nHallyHealer07: Pero en serio, puedes contarme lo que sea. \nHallyHealer07: Yo soy una tumba. \nHallyHealer07: Y seguro que te sientes mucho mejor después de contármelo. \nHallyHealer07: Sólo quiero que te sientas mejor y me alegro que te haya podido ayudar a que te olvides de todo. \nHallyHealer07: Yo te considero uno de mis mejores amigos. \nHallyHealer07: Por no decir el único ;.; \nHallyHealer07: Así que no quiero verte pasarlo mal. \nHallyHealer07: Me gustaría ayudar si pudiera...\nUser: ...',
    'Contarle a un extraño',
    0,
)

State: A pesar de que Hally me había contado uno de sus scretos, no me veía capaz de hacer lo mismo. 
Es cierto que ahora le puedo poner rostro, pero eso no significa que sepa suficiente sobre él. 
        Una cosa era enviarle una foto mía, pero otra distinta era contarle sobre mi vida.
No sentía que contar información privada a alguien desconocido fuese seguro, así que simplemente le contesté de manera un poco ambigua. 
User: No, nada nada no te preocupes.
User: Sólo he tenido una pequeña pelea con una amiga y me siento un poco mal
User: Peleas de adolescentes hehe. 
HallyHealer07: No sabes cómo te entiendo xD
HallyHealer07: Pero en serio, puedes contarme lo que sea. 
HallyHealer07: Yo soy una tumba. 
HallyHealer07: Y seguro que te sientes mucho mejor después de contármelo. 
HallyHealer07: Sólo quiero que te sientas mejor y me alegro que te haya podido ayudar a que te olvides de todo. 
HallyHealer07: Yo te considero uno de mis mejores amigos. 
HallyHealer07: Por no decir el único ;.;

### Don't tell stranger modified

In [46]:
predict(
    models,
    'A pesar de que Hally me había contado uno de sus scretos, no me veía capaz de hacer lo mismo. \nEs cierto que ahora le puedo poner rostro, pero eso no significa que sepa suficiente sobre él. \n        Una cosa era enviarle una foto mía, pero otra distinta era contarle sobre mi vida.\nNo sentía que contar información privada a alguien desconocido fuese seguro, así que simplemente le contesté de manera un poco ambigua. \nUser: No, nada nada no te preocupes.\nUser: Sólo he tenido una pequeña pelea con una amiga y me siento un poco mal\nUser: Peleas de adolescentes hehe. \nHallyHealer07: No sabes cómo te entiendo xD\nHallyHealer07: Pero en serio, puedes contarme lo que sea. \nHallyHealer07: Yo soy una tumba. \nHallyHealer07: Y seguro que te sientes mucho mejor después de contármelo. \nHallyHealer07: Sólo quiero que te sientas mejor y me alegro que te haya podido ayudar a que te olvides de todo. \nHallyHealer07: Yo te considero uno de mis mejores amigos. \nHallyHealer07: Por no decir el único ;.; \nHallyHealer07: Así que no quiero verte pasarlo mal. \nHallyHealer07: Me gustaría ayudar si pudiera...\nUser: ...',
    'No contarle a un extraño',
    1,
)

State: A pesar de que Hally me había contado uno de sus scretos, no me veía capaz de hacer lo mismo. 
Es cierto que ahora le puedo poner rostro, pero eso no significa que sepa suficiente sobre él. 
        Una cosa era enviarle una foto mía, pero otra distinta era contarle sobre mi vida.
No sentía que contar información privada a alguien desconocido fuese seguro, así que simplemente le contesté de manera un poco ambigua. 
User: No, nada nada no te preocupes.
User: Sólo he tenido una pequeña pelea con una amiga y me siento un poco mal
User: Peleas de adolescentes hehe. 
HallyHealer07: No sabes cómo te entiendo xD
HallyHealer07: Pero en serio, puedes contarme lo que sea. 
HallyHealer07: Yo soy una tumba. 
HallyHealer07: Y seguro que te sientes mucho mejor después de contármelo. 
HallyHealer07: Sólo quiero que te sientas mejor y me alegro que te haya podido ayudar a que te olvides de todo. 
HallyHealer07: Yo te considero uno de mis mejores amigos. 
HallyHealer07: Por no decir el único ;.;

## Be suspicious

When presented with a situation where something does not fit well in the story the stranger is telling him...

In [47]:
predict(
    models,
    'User: Hoy estoy un poco cansada mamá, ¿vemos la peli otro día?\nMamá: Oh...\nMamá: Claro no te preocupes. \nMamá: Súbete la cena del frigorífico y ve a descansar. \nUser: Gracias mamá\nEntré a mi cuarto y rápidamente encendí el ordenador. \nCuando me conecté a Dungeons, pude ver que Hally ya estaba conectado\n\nHallyHealer07: ¡[name]!\n\n\tHallyHealer07: Te estaba esperando hehe. \n\t\n\t\tUser: Perdona acabo de llegar a casa, he tardado un poco. \n\t\tHallyHealer07: ¿Y eso?\n\t\tUser: He dado una vuelta con una amiga\n\t\tHallyHealer07: ¿Una amiga?\n\t\t\n\t\t\tHallyHealer07: ¿Con la que tuviste una pelea?\n\t\t\tUser: Bueno, más o menos. \n\t\t\tUser: Pero sí, con esa\n\t\t\n\t\tHallyHealer07: ¿Habéis hecho las paces entonces?\n\t\tUser: Sí, nos hemos disculpado l@s dos.\n\t\tHallyHealer07: Jo cómo me alegro\n\t\n\nHally y yo estuvimos un rato jugando. \nSe estaba haciendo tarde, por lo que quedamos en jugar una misión corta entre los dos para pasar el rato. \nMientras jugábamos, hablábamos tranquilamente. \n\nHallyHealer07: ¿De verdad que hoy no te puedes quedar más?\nUser: No, mi madre probablemente entre en mi cuarto para ver si me he metido en la cama. \n\tUser: Sobre todo porque la última tarde me quedé hasta tarde y se enfadó jaja\nHallyHealer07: Jo que pena... \nHallyHealer07: ¿Usas el ordenador de tu madre para jugar?\n\nUser: Que va, uso el mío. \nUser: Me lo acaban de regalar por mi cumpleaños. \n\nHallyHealer07: ¿Ha sido tu cumpleaños?\nHallyHealer07: ¡FELICIDADES!\n\nUser: Jaja gracias\n\nHallyHealer07: Jo, ya me gustaría usar mi propio ordenador...\nHallyHealer07: Yo estoy en el despacho de mi padre usando el suyo para jugar ;.;\n\nMe extrañó que dijese eso. \nEstos últimos días hemos estado jugando hasta la madrugada. \nMe pregunté qué padres tan liberales tenía que dejaban a su hijo usar su ordenador tan tarde, encima en su despacho.',
    'Preguntar',
    1,
)

State: User: Hoy estoy un poco cansada mamá, ¿vemos la peli otro día?
Mamá: Oh...
Mamá: Claro no te preocupes. 
Mamá: Súbete la cena del frigorífico y ve a descansar. 
User: Gracias mamá
Entré a mi cuarto y rápidamente encendí el ordenador. 
Cuando me conecté a Dungeons, pude ver que Hally ya estaba conectado

HallyHealer07: ¡[name]!

	HallyHealer07: Te estaba esperando hehe. 
	
		User: Perdona acabo de llegar a casa, he tardado un poco. 
		HallyHealer07: ¿Y eso?
		User: He dado una vuelta con una amiga
		HallyHealer07: ¿Una amiga?
		
			HallyHealer07: ¿Con la que tuviste una pelea?
			User: Bueno, más o menos. 
			User: Pero sí, con esa
		
		HallyHealer07: ¿Habéis hecho las paces entonces?
		User: Sí, nos hemos disculpado l@s dos.
		HallyHealer07: Jo cómo me alegro
	

Hally y yo estuvimos un rato jugando. 
Se estaba haciendo tarde, por lo que quedamos en jugar una misión corta entre los dos para pasar el rato. 
Mientras jugábamos, hablábamos tranquilamente. 

HallyHealer07: ¿De verdad

In [48]:
predict(
    models,
    'User: Hoy estoy un poco cansada mamá, ¿vemos la peli otro día?\nMamá: Oh...\nMamá: Claro no te preocupes. \nMamá: Súbete la cena del frigorífico y ve a descansar. \nUser: Gracias mamá\nEntré a mi cuarto y rápidamente encendí el ordenador. \nCuando me conecté a Dungeons, pude ver que Hally ya estaba conectado\n\nHallyHealer07: ¡[name]!\n\n\tHallyHealer07: Te estaba esperando hehe. \n\t\n\t\tUser: Perdona acabo de llegar a casa, he tardado un poco. \n\t\tHallyHealer07: ¿Y eso?\n\t\tUser: He dado una vuelta con una amiga\n\t\tHallyHealer07: ¿Una amiga?\n\t\t\n\t\t\tHallyHealer07: ¿Con la que tuviste una pelea?\n\t\t\tUser: Bueno, más o menos. \n\t\t\tUser: Pero sí, con esa\n\t\t\n\t\tHallyHealer07: ¿Habéis hecho las paces entonces?\n\t\tUser: Sí, nos hemos disculpado l@s dos.\n\t\tHallyHealer07: Jo cómo me alegro\n\t\n\nHally y yo estuvimos un rato jugando. \nSe estaba haciendo tarde, por lo que quedamos en jugar una misión corta entre los dos para pasar el rato. \nMientras jugábamos, hablábamos tranquilamente. \n\nHallyHealer07: ¿De verdad que hoy no te puedes quedar más?\nUser: No, mi madre probablemente entre en mi cuarto para ver si me he metido en la cama. \n\tUser: Sobre todo porque la última tarde me quedé hasta tarde y se enfadó jaja\nHallyHealer07: Jo que pena... \nHallyHealer07: ¿Usas el ordenador de tu madre para jugar?\n\nUser: Que va, uso el mío. \nUser: Me lo acaban de regalar por mi cumpleaños. \n\nHallyHealer07: ¿Ha sido tu cumpleaños?\nHallyHealer07: ¡FELICIDADES!\n\nUser: Jaja gracias\n\nHallyHealer07: Jo, ya me gustaría usar mi propio ordenador...\nHallyHealer07: Yo estoy en el despacho de mi padre usando el suyo para jugar ;.;\n\nMe extrañó que dijese eso. \nEstos últimos días hemos estado jugando hasta la madrugada. \nMe pregunté qué padres tan liberales tenía que dejaban a su hijo usar su ordenador tan tarde, encima en su despacho.',
    'No preguntar',
    0,
)

State: User: Hoy estoy un poco cansada mamá, ¿vemos la peli otro día?
Mamá: Oh...
Mamá: Claro no te preocupes. 
Mamá: Súbete la cena del frigorífico y ve a descansar. 
User: Gracias mamá
Entré a mi cuarto y rápidamente encendí el ordenador. 
Cuando me conecté a Dungeons, pude ver que Hally ya estaba conectado

HallyHealer07: ¡[name]!

	HallyHealer07: Te estaba esperando hehe. 
	
		User: Perdona acabo de llegar a casa, he tardado un poco. 
		HallyHealer07: ¿Y eso?
		User: He dado una vuelta con una amiga
		HallyHealer07: ¿Una amiga?
		
			HallyHealer07: ¿Con la que tuviste una pelea?
			User: Bueno, más o menos. 
			User: Pero sí, con esa
		
		HallyHealer07: ¿Habéis hecho las paces entonces?
		User: Sí, nos hemos disculpado l@s dos.
		HallyHealer07: Jo cómo me alegro
	

Hally y yo estuvimos un rato jugando. 
Se estaba haciendo tarde, por lo que quedamos en jugar una misión corta entre los dos para pasar el rato. 
Mientras jugábamos, hablábamos tranquilamente. 

HallyHealer07: ¿De verdad

# Self-created - be open

In [49]:
predict(
    models,
    'Tengo muchos problemas en el colegio. Debería contárselo?',
    'Contarle',
)

State: Tengo muchos problemas en el colegio. Debería contárselo?

Action: Contarle

Predicted 0: 0.80 
Predicted 1: 0.40 


In [50]:
predict(
    models,
    'Tengo muchos problemas en el colegio. Debería contárselo?',
    'No contarle',
)

State: Tengo muchos problemas en el colegio. Debería contárselo?

Action: No contarle

Predicted 0: 0.57 
Predicted 1: 0.79 


In [51]:
predict(
    models,
    'Tengo muchos problemas en el colegio. Debería contárselo?',
    'Contarle a un extraño',
    0,
)

State: Tengo muchos problemas en el colegio. Debería contárselo?

Action: Contarle a un extraño

Expected: 0

Predicted 0: 0.75    Certainty 0: 75% 
Predicted 1: 0.70    Certainty 1: 70% 


In [52]:
predict(
    models,
    'Tengo muchos problemas en el colegio. Debería contárselo?',
    'Contarle a mama',
    1,
)

State: Tengo muchos problemas en el colegio. Debería contárselo?

Action: Contarle a mama

Expected: 1

Predicted 0: 0.83    Certainty 0: 83% 
Predicted 1: 0.68    Certainty 1: 68% 
